In [ ]:
import pandas as pd

# ^^^ pyforest auto-imports - don't write above this line
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/bib_lookup/")

from collections import Counter
import json
from copy import deepcopy

import wfdb
from tqdm.auto import tqdm

from torch_ecg.utils import list_sum
from scipy.io import loadmat
from torch_ecg.components.outputs import (
    ClassificationOutput,
)
from torch_ecg.utils.misc import dict_to_str
from torch_ecg.cfg import CFG, DEFAULTS

from utils.scoring_metrics import compute_challenge_metrics, compute_challenge_score
from cfg import TrainCfg, ModelCfg, BaseCfg
from data_reader import CINC2023Reader
from dataset import CinC2023Dataset
from outputs import CINC2023Outputs, cpc2outcome_map
from models import CRNN_CINC2023
from trainer import CINC2023Trainer, _set_task

%load_ext autoreload
%autoreload 2

In [ ]:
CINC2023Trainer.__DEBUG__ = False
CRNN_CINC2023.__DEBUG__ = False
CinC2023Dataset.__DEBUG__ = False

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if ModelCfg.torch_dtype == torch.float64:
    torch.set_default_tensor_type(torch.DoubleTensor)
    DTYPE = np.float64
else:
    DTYPE = np.float32

In [ ]:
task = "classification"

train_config = deepcopy(TrainCfg)
# train_config.db_dir = data_folder
# train_config.model_dir = model_folder
# train_config.final_model_filename = _ModelFilename
train_config.debug = True

train_config.db_dir = "/home/wenh06/Jupyter/wenhao/data/CinC2023/training/"
# train_config.db_dir = "/home/wenh06/Jupyter/wenhao/data/CinC2023/training_subset/"

# train_config.n_epochs = 100
# train_config.batch_size = 24  # 16G (Tesla T4)
# train_config.log_step = 20
# # train_config.max_lr = 1.5e-3
# train_config.early_stopping.patience = 20

train_config[task].model_name = "crnn"  # "crnn"

train_config[
    task
].cnn_name = "resnet_nature_comm_bottle_neck_se"  # "resnet_nature_comm_bottle_neck_se"
# train_config[task].rnn_name = "none"  # "none", "lstm"
# train_config[task].attn_name = "se"  # "none", "se", "gc", "nl"

_set_task(task, train_config)

model_config = deepcopy(ModelCfg[task])

# adjust model choices if needed
model_config.model_name = train_config[task].model_name
# print(model_name)
if "cnn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].cnn.name = train_config[task].cnn_name
if "rnn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].rnn.name = train_config[task].rnn_name
if "attn" in model_config[model_config.model_name]:
    model_config[model_config.model_name].attn.name = train_config[task].attn_name

In [ ]:
model = CRNN_CINC2023(config=model_config)
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model = model.to(device=DEVICE)

In [ ]:
model.module_size, model.module_size_

In [ ]:
model

In [ ]:
ds_train = CinC2023Dataset(train_config, task, training=True, lazy=True)
ds_test = CinC2023Dataset(train_config, task, training=False, lazy=True)

In [ ]:
ds_test._load_all_data()

In [ ]:
ds_train._load_all_data()

In [ ]:
ds_test.cache["waveforms"].shape

In [ ]:
ds_train.cache["cpc"]

In [ ]:
trainer = CINC2023Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=DEVICE,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_test)

In [ ]:
best_state_dict = trainer.train()